In [ ]:
import collections
import collections.abc
collections.Callable = collections.abc.Callable

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [ ]:
# 셀레니움 드라이버 생성
def create_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

# 키워드로 상위 5개 문서 검색
def get_article_urls_from_keyword(keyword, max_items=5):
    driver = create_driver()
    base_url = "https://encykorea.aks.ac.kr"
    driver.get(base_url)
    time.sleep(1)

    search_input = driver.find_element(By.ID, "keyword")
    search_input.clear()
    search_input.send_keys(keyword)
    driver.find_element(By.CLASS_NAME, "main-search").click()
    time.sleep(2)

    items = driver.find_elements(By.CSS_SELECTOR, "ul.encyclopedia-list li.item")
    results = []
    for item in items[:max_items]:
        try:
            link_tag = item.find_element(By.CSS_SELECTOR, "a")
            title_tag = item.find_element(By.CSS_SELECTOR, ".title")
            href = link_tag.get_attribute("href")
            title = title_tag.text.strip()
            results.append((title, href))
        except:
            continue

    driver.quit()
    return results

# 본문 파싱
def parse_article_contents(title, url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        content = soup.find("div", class_="contents-detail-contents")
        if not content:
            return {"제목": title, "URL": url, "정의": "본문 없음"}

        sections = content.find_all("div", class_="detail-section")
        result = {"제목": title, "URL": url}
        for sec in sections:
            title_tag = sec.find(class_="section-title")
            body_tag = sec.find(class_="section-body")
            if title_tag and body_tag:
                key = title_tag.get_text(strip=True)
                value = body_tag.get_text(separator="\n").strip()
                result[key] = value

        return result
    except Exception as e:
        return {"제목": title, "URL": url, "정의": f" 오류: {e}"}

# 원본 데이터 기반 크롤링 수행
def crawl_articles_from_excel_rows(excel_path, max_keywords=5, articles_per_keyword=5):
    df = pd.read_excel(excel_path)
    all_results = []

    for idx, row in df.iterrows():
        row_info = {
            "열1": row.get("열1", ""),
            "열2": row.get("열2", ""),
            "열3": row.get("열3", ""),
            "열4": row.get("열4", "")
        }

        keywords = str(row_info["열4"]).split(',') if pd.notna(row_info["열4"]) else []
        for kw in keywords[:max_keywords]:
            kw = kw.strip()
            if not kw:
                continue

            print(f"({idx}) 키워드 '{kw}' 문서 검색 중...")
            articles = get_article_urls_from_keyword(kw, articles_per_keyword)
            if not articles:
                result = row_info.copy()
                result.update({"키워드": kw, "제목": "문서 없음", "URL": "", "정의": "N/A"})
                all_results.append(result)
                continue

            for title, url in articles:
                parsed = parse_article_contents(title, url)
                result = row_info.copy()
                result["키워드"] = kw
                result.update(parsed)
                all_results.append(result)
                time.sleep(1)

    return pd.DataFrame(all_results)

In [ ]:
excel_path = "교과서 판서 별 키워드.xlsx"  # 실제 파일명
df_final = crawl_articles_from_excel_rows(excel_path, max_keywords=2, articles_per_keyword=5)

df_final.to_excel("교과서_키워드별_문서결과_5개씩.xlsx", index=False)
print("✅ 저장 완료: 교과서_키워드별_문서결과_5개씩.xlsx")

🔍 (0) 키워드 '서역' 문서 검색 중...
🔍 (0) 키워드 '중국' 문서 검색 중...
🔍 (1) 키워드 '서역' 문서 검색 중...
🔍 (1) 키워드 '중국' 문서 검색 중...
🔍 (2) 키워드 '일본' 문서 검색 중...
🔍 (2) 키워드 '교류' 문서 검색 중...
🔍 (3) 키워드 '일본' 문서 검색 중...
🔍 (3) 키워드 '교류' 문서 검색 중...
🔍 (4) 키워드 '일본' 문서 검색 중...
🔍 (4) 키워드 '교류' 문서 검색 중...
🔍 (5) 키워드 '일본' 문서 검색 중...
🔍 (5) 키워드 '고류' 문서 검색 중...
🔍 (6) 키워드 '일본' 문서 검색 중...
🔍 (6) 키워드 '교류' 문서 검색 중...
🔍 (7) 키워드 '일본' 문서 검색 중...
🔍 (7) 키워드 '교류' 문서 검색 중...
🔍 (8) 키워드 '수나라' 문서 검색 중...
🔍 (8) 키워드 '중국 통일' 문서 검색 중...
🔍 (9) 키워드 '수나라' 문서 검색 중...
🔍 (9) 키워드 '중국 통일' 문서 검색 중...
🔍 (10) 키워드 '수나라' 문서 검색 중...
🔍 (10) 키워드 '중국 통일' 문서 검색 중...
🔍 (11) 키워드 '수나라' 문서 검색 중...
🔍 (11) 키워드 '중국 통일' 문서 검색 중...
🔍 (12) 키워드 '수나라' 문서 검색 중...
🔍 (12) 키워드 '고구려 침략' 문서 검색 중...
🔍 (13) 키워드 '수나라' 문서 검색 중...
🔍 (13) 키워드 '고구려 침략' 문서 검색 중...
🔍 (14) 키워드 '수나라' 문서 검색 중...
🔍 (14) 키워드 '고구려 침략' 문서 검색 중...
🔍 (15) 키워드 '수나라' 문서 검색 중...
🔍 (15) 키워드 '고구려 침략' 문서 검색 중...
🔍 (16) 키워드 '당나라' 문서 검색 중...
🔍 (16) 키워드 '고구려 침략' 문서 검색 중...
🔍 (17) 키워드 '당나라' 문서 검색 중...
🔍 (17) 키워드 '고구려 침략' 문서 검색 중...
🔍 